<a href="https://colab.research.google.com/github/deeponcology/PyTorchMedicalAI/blob/master/shlomo_dl_0002_tensors_collab_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning in Medical AI 2018/2019 using PyTorch & Google Collab.


<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/drive/1JEIeD_445sFvcjSrITB5Z_oW8VHRS_kA">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/deeponcology/PyTorchMedicalAI/blob/master/shlomo_dl_0001_cuda_collab_pytorch.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

<img src="https://github.com/deeponcology/PyTorchMedicalAI/raw/master/assets/tumor_visdom.jpg" align="center" width=30%>

### Author: 
***Shlomo Kashani***, Head of AI at www.DeepOncology.AI, shlomo@deeponcology.ai 

<img src="https://github.com/deeponcology/PyTorchMedicalAI/raw/master/assets/line-up-small.png" align="center" width=30%>


### Synopsys:
This is the hands-on deep learning tutorial series for the 2018/2019 Medical AI course. The series will guide you through the most basic building blocks such as installing CUDA to training advanced CNN's such as SeNet. 

### DataSets:
We foster the use of Medical Data Sets (https://grand-challenge.org/All_Challenges/) and predominantly those available (but not only) via Kaggle.

### About PyTorch:

PyTorch is an open source library for numerical computation using  computation graphs. Nodes in the graph represent mathematical operations, while the graph edges represent the multidimensional data arrays (tensors) communicated between them. 


Similar to python programming, we can add and execute a node to the computation graph immediately. This property makes it easy to debug the code and inspect the values in the network.

### The other Notebooks in this series: 
If you are familiar with these topics, feel free to jump to other modules.



# Collab notebook: 002 using PyTorch Tensors

In [0]:
%reset -f
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
import sys
sys.version

# !pip3 install torch==0.4
# !pip3 install torchvision

!pip3 install 'torch==0.4.0'
!pip3 install 'torchvision==0.2.1'
!pip3 install --no-cache-dir -I 'pillow==5.1.0'

# Restart Kernel
# This workaround is needed to properly upgrade PIL on Google Colab.
import os
os._exit(00)


    100% |████████████████████████████████| 484.0MB 28kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5c044000 @  0x7fd51cb612a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 2.7MB/s 
    100% |████████████████████████████████| 2.0MB 9.0MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 2.0MB 17.6MB/s 


## Import PyTorch once again

In [1]:
import matplotlib.pyplot as plt
import time
from shutil import copyfile
from os.path import isfile, join, abspath, exists, isdir, expanduser
from os import listdir, makedirs, getcwd, remove
from PIL import Image
from mpl_toolkits.axes_grid1 import ImageGrid
import pandas as pd
import numpy as np
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as func
import torchvision
from torchvision import transforms, datasets, models
import random 

import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

use_cuda = torch.cuda.is_available()
# use_cuda = False

print("USE CUDA=" + str (use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

manualSeed = 2222
def fixSeed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


if manualSeed is None:
        manualSeed = 999
fixSeed(manualSeed)

__Python VERSION: 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
__pyTorch VERSION: 0.4.0
__CUDA VERSION
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148
__CUDNN VERSION: 7102
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0
Available devices  1
Current cuda device  0
USE CUDA=True


# Numpy vs PyTorch Syntax

Numpy	           /  Pytorch

- np.zeros((2, 3))	    torch.zeros(2,3)
- np.random.rand(2, 3)	torch.rand(2,3)
- x.reshape(1, -1)	    x.view(1, -1)
- x.shape	                x.size()
- x.dot(w)	            x.mm(w)
- x.matmul(w)	            x.bmm(w)
- x.T	                    x.t()
- x.transpose(0, 2, 1)	x.permute(0, 2, 1)
- x.argmax(axis=1)	    _, i = x.max(dim=1)
- np.sum(x, axis=1)	    torch.sum(x, dim=1)
- np.maxium(x, 0)	        torch.clamp(x, min=0)
- x.clone()	            x.copy()

## Torch Tensors


In [4]:
from __future__ import print_function
import torch
from torch.autograd import Variable


import os, sys, pdb, shutil, time, random
import argparse
import torch
import torch.backends.cudnn as cudnn
import torchvision.datasets as dset
import torchvision.transforms as transforms
# from utils import AverageMeter, RecorderMeter, time_string, convert_secs2time
# from tensorboard_logger import configure, log_value
from tqdm import tqdm
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as func
import torchvision
from torchvision import transforms, datasets, models
import random
from shutil import copyfile
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import time
from shutil import copyfile
from os.path import isfile, join, abspath, exists, isdir, expanduser
from os import listdir, makedirs, getcwd, remove
from mpl_toolkits.axes_grid1 import ImageGrid
import pandas as pd
import numpy as np
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as func
import torchvision
from torchvision import transforms, datasets, models
import random
import sys
from glob import glob
import fnmatch


x=torch.Tensor(3,2)
print (type(x))
print (x)

# how variables work
x = Variable(x)
print ("x:" + str (x))
print ("requires grad:" + str(x.requires_grad))
print ("data:" + str(x.data))

<class 'torch.Tensor'>
tensor(1.00000e-36 *
       [[ 2.6350,  0.0000],
        [ 0.0000,  0.0000],
        [    nan,  0.0000]])
x:tensor(1.00000e-36 *
       [[ 2.6350,  0.0000],
        [ 0.0000,  0.0000],
        [    nan,  0.0000]])
requires grad:False
data:tensor(1.00000e-36 *
       [[ 2.6350,  0.0000],
        [ 0.0000,  0.0000],
        [    nan,  0.0000]])


In [0]:
import os
from glob import glob
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image


In [5]:
x=torch.rand(3,4)
print (type(x))
print (x)

<class 'torch.Tensor'>
tensor([[ 0.1632,  0.6885,  0.9803,  0.1763],
        [ 0.4946,  0.6567,  0.1276,  0.5089],
        [ 0.6165,  0.0302,  0.3607,  0.1505]])


In [6]:
print (x[1:])

tensor([[ 0.4946,  0.6567,  0.1276,  0.5089],
        [ 0.6165,  0.0302,  0.3607,  0.1505]])


In [7]:
x.numpy()

array([[0.1632443 , 0.6884852 , 0.9802843 , 0.17633855],
       [0.49460745, 0.65665674, 0.12756538, 0.5089476 ],
       [0.61649853, 0.03015894, 0.36068344, 0.15046859]], dtype=float32)

In [8]:
if torch.cuda.is_available():
    x = x.cuda()*2 
    
print (type(x))
print (x)    

<class 'torch.Tensor'>
tensor([[ 0.3265,  1.3770,  1.9606,  0.3527],
        [ 0.9892,  1.3133,  0.2551,  1.0179],
        [ 1.2330,  0.0603,  0.7214,  0.3009]], device='cuda:0')
